## pós-Processamento de dados de GCMS de Óleos Essenciais com Índice de Kovats (mistura de hidrocarbonetos)

#### INTRODUÇÃO

O pipeline KIMSA (Kovats Index and Mass Spectral Annotation) foi desenvolvido para integrar abordagens complementares de identificação de compostos em óleos essenciais analisados por cromatografia gasosa acoplada à espectrometria de massas (GC-MS). O método combina a anotação baseada no índice de Kovats — calculado a partir dos tempos de retenção experimentais em comparação com uma série de alcanos — com a identificação espectral por similaridade de cosseno entre os espectros experimentais e uma biblioteca de referência.

A integração dessas duas abordagens proporciona maior robustez e confiabilidade na identificação de compostos, permitindo verificar a consistência entre o comportamento cromatográfico (via índice de retenção) e o perfil espectral (via fragmentação de massa). O pipeline foi projetado para processar automaticamente arquivos no formato .mgf (espectros experimentais) e .msp (biblioteca), gerar anotações múltiplas acima de um limiar de similaridade, identificar redundâncias e produzir uma tabela consolidada para fins de análise e geração de laudos cromatográficos.

##### Processamento no MZMine 3.9.0
1. Detecção de massa (Mass Detection)
2. Construção de cromatogramas (ADAP Chromatogram Builder)
3. Deconvolução de picos cromatográficos (ADAP Peak Picking)
4. Clusterização hierárquica (ADAP Hierarchical Clustering)

##### Sugestão inicial (a ser otimizada)
###### A partir do arquivo BATCH: `Essential_Oils_Batch.xml`

| **Etapa**                   | **Parâmetro**                           | **Valor**                      | **Descrição**                                                   |
|-----------------------------|----------------------------------------|--------------------------------|-----------------------------------------------------------------|
| **Mass Detection**           | Detector                                | Centroid                       | Detector de picos centroidizados                                |
|                             | Noise level                             | 10000.0                        | Limite mínimo de intensidade para detectar um pico              |
|                             | Isótopos abaixo do noise level          | Ativado                        | Permite isótopos mesmo abaixo do limite de ruído               |
|                             | Elementos para isótopos                 | H, C, N, O, S                  | Elementos considerados para detecção isotópica                  |
|                             | Tolerância m/z isótopos                 | 0.5 Da ou 0.0 ppm            | Tolerância para agrupamento de isótopos                         |
| **Chromatogram Builder**     | Mínimo de scans consecutivos            | 12                             | Número mínimo de pontos contínuos no cromatograma               |
|                             | Intensidade mínima por scan              | 1000                           | Intensidade mínima para contar um scan válido                   |
|                             | Altura mínima absoluta                  | 10000                          | Altura mínima do pico cromatográfico                            |
|                             | Tolerância m/z entre scans              | 0.5 Da                         | Tolerância de variação de m/z entre pontos                      |
| **ADAP Peak Picking**:      | S/N threshold                           | 5.0                            | Relação sinal/ruído mínima para detecção                        |
|                             | Mínima altura do pico                   | 1000                           | Altura mínima de um pico para ser considerado                   |
|                             | Coeficiente/área mínimo                 | 50                             | Mínimo coeficiente/área                                        |
|                             | Intervalo de tempo do pico              | 0 - 10 min                     | Intervalo aceito para duração do pico                           |
|                             | Pareamento com espectros MS/MS          | Ativado                        | Permite pareamento com espectros MS/MS                          |
|                             | Tolerância precursor m/z                | 0.01 Da / 10 ppm               | Tolerância para pareamento de precursor                         |
|                             | Filtro por tempo de retenção            | 0.2 min                        | Filtro de tempo de retenção                                     |
| **ADAP Hierarchical Clustering**     | Distância mínima de cluster             | 0.01                           | Distância mínima para formar um cluster                         |
|                             | Tamanho mínimo de cluster               | 2                              | Número mínimo de picos para formar um cluster                   |
|                             | Intensidade mínima do cluster           | 500                            | Intensidade mínima de um cluster                                |
|                             | Tolerância de similaridade de forma     | 60.0                           | Similaridade mínima para agrupar picos pela forma               |

* Não alinhar os dados entre as amostras. Cada amostra terá um conjunto de 3 arquivos exportados: `_quant.csv`,  `.mgf`  e `_Chromatogram.xlsx`

##### Requisitos de entrada (arquivos):
| **Arquivo**              | **Formato** | **Origem**          | **Descrição**                                                                                           |
| ------------------------ | ----------- | ------------------- | ------------------------------------------------------------------------------------------------------- |
| Cromatograma  | `_Chromatogram.xlsx`      | Exportado do MZMine | Contém os _cromatogramas_ de massa experimentais no formato Excel (MS1).             |
| Espectros experimentais  | `.mgf`      | Exportado do MZMine | Contém os _espectros de massa_ experimentais no formato Mascot Generic Format (MS1 ou MS/MS).             |
| Tabela de picos          | `_quant.csv`      | Exportado do MZMine | Contém `row ID`, `row m/z`, `row retention time`, área de pico, e outras informações quantitativas.     |
| Tabela de alcanos        | `.csv`      | Experimental        | Tabela de referência com número de carbonos e tempos de retenção dos alcanos (para cálculo de Kovats).  |
| Biblioteca de RI de referência | `RI_Library.csv`      | Biblioteca externa  | Arquivo contendo dados de índices de retenção de referência de substâncias conhecidas. |
| Biblioteca de referência | `Essential_Oils_MS1Library_1.msp`      | Biblioteca externa| Arquivo contendo espectros de referência no formato NIST MSP, com `Name`, `Num Peaks` e lista de picos. |


⚠️ Observação: os espectros nos arquivos .mgf e .msp devem estar em formato tabular, com listas de pares m/z intensidade.

#### Importação de bibliotecas e leitura de arquivos

Nesta etapa, são importadas as bibliotecas necessárias para manipulação de dados, processamento de espectros e visualização. Em seguida, os arquivos contendo os espectros experimentais (.mgf), a biblioteca de referência (.msp) e os dados de Kovats são carregados para o notebook.

In [1]:
# 1️⃣ IMPORTAÇÃO DE BIBLIOTECAS
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import sys
import os
from docxtpl import DocxTemplate
from docxtpl import InlineImage
from docx.shared import Cm

sys.path.append(os.path.abspath('..'))  # sobe um nível de pasta
import essential_oils_kovats_processing as eokp

#### Inspeção inicial dos arquivos

Aqui são exibidas as primeiras linhas dos arquivos .mgf e .msp para confirmar sua estrutura e verificar se os dados foram lidos corretamente. Essa inspeção é fundamental para garantir que o parser funcione de acordo com o formato dos arquivos.

In [2]:
# 2️⃣ LEITURA DOS ARQUIVOS

# 📝 Caminho da pasta contendo os arquivos
input_dir = r"C:\Users\borge\Documents\gcms_essentialoils_Kovats\batch"  # substitua pelo seu caminho

# 📄 Arquivo fixo de RTs dos alcanos
file_hidrocarbon_rts = Path("..") / "resources" / "Hidrocarbon_RTs.csv"
file_ri_library = Path("..") / "resources" / "RI_Library.csv"

df_hidrocarbon, df_ri, sample_prefixes, dfs_quant, dfs_chrom = eokp.carregar_dados(
    input_dir=input_dir,
    file_hidrocarbon_rts=file_hidrocarbon_rts,
    file_ri_library=file_ri_library
)

✅ Tabela de RTs dos alcanos importada.
✅ Tabela de referência RI importada.
✅ Amostra GT_PadraoHidroc_13-06-2024 importada.
✅ Amostra GT_OE2-23_13-06-2024 importada.
✅ Amostra GT_OE3-23_13-06-2024 importada.
✅ Amostra GT_OE1-23_13-06-2024 importada.


#### Parsing dos espectros experimentais e da biblioteca de referência

Nesta etapa, os arquivos .mgf e .msp são processados e convertidos em dicionários contendo os espectros no formato {ID: [(m/z, intensidade), ...]}. Cada espectro é identificado por um FEATURE_ID ou Name, e suas listas de picos são armazenadas para posterior comparação.

In [3]:
# 3️⃣ PREPARAÇÃO DA TABELA DE ALCANOS
# Vamos extrair apenas as colunas que interessam: número de carbonos e RT em minutos
df_hidrocarbon = df_hidrocarbon.rename(columns=lambda x: x.strip())  # remove espaços
rt_column = 'Current RT in minutes' if 'Current RT in minutes' in df_hidrocarbon.columns else 'Typical RT in minutes'
df_alkanes = df_hidrocarbon[['Number of Carbon Atoms', rt_column]].dropna()
df_alkanes[rt_column] = pd.to_numeric(df_alkanes[rt_column], errors='coerce')
df_alkanes = df_alkanes.dropna().sort_values(rt_column)
print("RTs dos alcanos processados:")
#display(df_alkanes)

x_fit, y_fit, p_model, r_squared = eokp.fit_polynomial_trendline(
    df_alkanes,
    x_col='Number of Carbon Atoms',
    y_col=rt_column,
    degree=4)

print(f"Equação do polinômio: {p_model}")
print(f"R²: {r_squared:.4f}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_alkanes['Number of Carbon Atoms'], y=df_alkanes[rt_column], mode='markers', name='Dados'))
fig.add_trace(go.Scatter(x=x_fit, y=y_fit, mode='lines', name='Curva Ajustada'))

fig.update_layout(
    title="Curva de Retenção (Alcanos)",
    xaxis_title="Número de Carbonos",
    yaxis_title="Tempo de Retenção (min)"
)

eokp.save_plotly_figure(fig, filename="curva_retencao.html")

RTs dos alcanos processados:
Equação do polinômio:           4           3         2
0.001072 x - 0.07951 x + 2.092 x - 19.48 x + 60.27
R²: 0.9999
✅ Figura salva em: images\curva_retencao.html


In [4]:
# 7️⃣ PLOTAR CROMATOGRAMA (ATUALIZADO)
for prefix, df_chrom in dfs_chrom.items():
    eokp.plot_chromatogram_auto_multitrace(df_chrom, prefix)


🔍 Colunas disponíveis para GT_PadraoHidroc_13-06-2024:
0: GT_PadraoHidroc_13-06-2024.CDF
1: Unnamed: 1
2: Unnamed: 2
✅ Gráfico salvo como HTML em: images\GT_PadraoHidroc_13-06-2024_chromatogram.html

🔍 Colunas disponíveis para GT_OE2-23_13-06-2024:
0: GT_OE2-23_13-06-2024.CDF
1: Unnamed: 1
2: Unnamed: 2
✅ Gráfico salvo como HTML em: images\GT_OE2-23_13-06-2024_chromatogram.html

🔍 Colunas disponíveis para GT_OE3-23_13-06-2024:
0: GT_OE3-23_13-06-2024.CDF
1: Unnamed: 1
2: Unnamed: 2
✅ Gráfico salvo como HTML em: images\GT_OE3-23_13-06-2024_chromatogram.html

🔍 Colunas disponíveis para GT_OE1-23_13-06-2024:
0: GT_OE1-23_13-06-2024.CDF
1: Unnamed: 1
2: Unnamed: 2
✅ Gráfico salvo como HTML em: images\GT_OE1-23_13-06-2024_chromatogram.html


#### Cálculo do índice de Kovats e anotação preliminar

Nesta etapa, é calculado o índice de Kovats para cada pico experimental com base nos tempos de retenção dos alcanos de referência. Para cada índice calculado, é realizada uma busca na biblioteca de índices de Kovats para identificar compostos potenciais dentro de uma tolerância especificada. Essa anotação preliminar associa cada pico a possíveis compostos candidatos com base exclusivamente no comportamento cromatográfico.

In [5]:
# 4️⃣ IDENTIFICAÇÂO CONFORME ÍNDICE KOVATS
# Aplicar a todas as amostras
dfs_quant_processed = {}
for prefix in sample_prefixes:
    df_quant = dfs_quant[prefix]
    df_quant_processed = eokp.process_sample_kovats(
        df_quant,
        alkane_table=df_hidrocarbon,
        rt_column="Current RT in minutes",  # ajuste conforme sua coluna real
        ref_table=df_ri,
        tolerance=10
    )
    dfs_quant_processed[prefix] = df_quant_processed
    print(f"✅ Kovats calculado para amostra {prefix}")

✅ Kovats calculado para amostra GT_PadraoHidroc_13-06-2024
✅ Kovats calculado para amostra GT_OE2-23_13-06-2024
✅ Kovats calculado para amostra GT_OE3-23_13-06-2024
✅ Kovats calculado para amostra GT_OE1-23_13-06-2024


---

#### Leitura e definição das funções de processamento espectral

Esta etapa realiza a leitura inicial dos arquivos .mgf (espectros experimentais) e .msp (biblioteca de espectros de referência) e define as funções principais do pipeline. As funções incluem o parsing dos arquivos para estruturas de dados internas, o cálculo da similaridade de cosseno entre espectros experimentais e de referência, e a busca das melhores correspondências. Estas ferramentas são fundamentais para realizar a comparação espectral e gerar as anotações baseadas em espectrometria de massas.

In [6]:
WFEqfew

NameError: name 'WFEqfew' is not defined

In [7]:
# Arquivo .msp fixo
file_msp = r"C:\Users\borge\Desktop\MS_software\MZmine_3.9.0\essential_oils_test\Essential_Oils_MS1Library_1.msp"
spectra_lib = eokp.parse_msp(file_msp)
print(f"✅ Biblioteca {file_msp} parseada com {len(spectra_lib)} espectros")

### B) Criar dicionário para armazenar resultados por amostra
results_by_sample = {}

eokp.processar_mgf_amostras(
    sample_prefixes=sample_prefixes,
    input_dir=input_dir,
    spectra_lib=spectra_lib,
    results_by_sample=results_by_sample)

✅ Biblioteca C:\Users\borge\Desktop\MS_software\MZmine_3.9.0\essential_oils_test\Essential_Oils_MS1Library_1.msp parseada com 8 espectros

🔍 Processando C:\Users\borge\Documents\gcms_essentialoils_Kovats\batch\GT_PadraoHidroc_13-06-2024.mgf...
✅ 25 espectros lidos de C:\Users\borge\Documents\gcms_essentialoils_Kovats\batch\GT_PadraoHidroc_13-06-2024.mgf
✅ Resultados salvos em results\GT_PadraoHidroc_13-06-2024_matches.csv

🔍 Processando C:\Users\borge\Documents\gcms_essentialoils_Kovats\batch\GT_OE2-23_13-06-2024.mgf...
✅ 28 espectros lidos de C:\Users\borge\Documents\gcms_essentialoils_Kovats\batch\GT_OE2-23_13-06-2024.mgf
✅ Resultados salvos em results\GT_OE2-23_13-06-2024_matches.csv

🔍 Processando C:\Users\borge\Documents\gcms_essentialoils_Kovats\batch\GT_OE3-23_13-06-2024.mgf...
✅ 95 espectros lidos de C:\Users\borge\Documents\gcms_essentialoils_Kovats\batch\GT_OE3-23_13-06-2024.mgf
✅ Resultados salvos em results\GT_OE3-23_13-06-2024_matches.csv

🔍 Processando C:\Users\borge\Docu

In [9]:
for prefix, df in results_by_sample.items():
    print(f"\n🔬 Matches com Similaridade ≥ 0.9 para {prefix}")
    high_sim = df[df['Similarity'] >= 0.9]
    if not high_sim.empty:
        display(high_sim.head(5))
    else:
        print("Nenhum match com alta similaridade.")


🔬 Matches com Similaridade ≥ 0.9 para GT_PadraoHidroc_13-06-2024


,FEATURE_ID,Best Match Name,Similarity
2,1,(+)-(Z)-Limonene oxide,1.0
4,2,(+)-(Z)-Limonene oxide,0.9
16,5,(+)-(Z)-Limonene oxide,0.9
20,6,(+)-(Z)-Limonene oxide,0.9
24,7,(+)-(Z)-Limonene oxide,0.9



🔬 Matches com Similaridade ≥ 0.9 para GT_OE2-23_13-06-2024


,FEATURE_ID,Best Match Name,Similarity
3,2,myrcene,0.90
5,3,(+)-(E)-Limonene oxide,0.99
9,5,δ-3-carene,0.98
10,5,sabinene,0.97
11,6,(+-)-Linalool,0.92



🔬 Matches com Similaridade ≥ 0.9 para GT_OE3-23_13-06-2024


,FEATURE_ID,Best Match Name,Similarity
0,1,(+-)-Limonene,1.00
2,2,(+-)-Limonene,0.95
4,2,(+)-(Z)-Limonene oxide,0.90
5,2,δ-3-carene,0.98
7,2,sabinene,0.99



🔬 Matches com Similaridade ≥ 0.9 para GT_OE1-23_13-06-2024


,FEATURE_ID,Best Match Name,Similarity
0,1,δ-3-carene,0.96
1,1,sabinene,0.96
2,1,α-pinene,0.99
3,2,(+-)-Linalool,0.91
5,2,δ-3-carene,0.98


#### Combinação das anotações por índice de Kovats e espectrometria de massas

Nesta etapa, os resultados da anotação por índice de Kovats são integrados com os resultados da anotação espectral, permitindo uma visão consolidada para cada row ID. A operação de merge garante que as informações experimentais (m/z, tempo de retenção), o índice de Kovats, a anotação por Kovats e as correspondências espectrais (nome e similaridade) fiquem reunidas em uma única tabela. O resultado é exibido no notebook e exportado para um arquivo Excel para posterior análise.

##### Exportação dos resultados finais

A tabela consolidada contendo as anotações combinadas é exportada para um arquivo Excel, permitindo análise externa, conferência manual ou integração em relatórios de qualidade.

In [12]:
eokp.processar_fusao_amostras(
    sample_prefixes=sample_prefixes,
    dfs_quant_processed=dfs_quant_processed,
    results_by_sample=results_by_sample,
    prefix_ignorado="GT_PadraoHidroc_13-06-2024"
)

⚠️ Ignorando amostra padrão GT_PadraoHidroc_13-06-2024

🔍 Processando fusão para GT_OE2-23_13-06-2024...
✅ Fusão completa para GT_OE2-23_13-06-2024
✅ Arquivo salvo: results\GT_OE2-23_13-06-2024_Identificacao_Combinada.xlsx

🔍 Processando fusão para GT_OE3-23_13-06-2024...
✅ Fusão completa para GT_OE3-23_13-06-2024
✅ Arquivo salvo: results\GT_OE3-23_13-06-2024_Identificacao_Combinada.xlsx

🔍 Processando fusão para GT_OE1-23_13-06-2024...
✅ Fusão completa para GT_OE1-23_13-06-2024
✅ Arquivo salvo: results\GT_OE1-23_13-06-2024_Identificacao_Combinada.xlsx

🎉 Processamento de todas as amostras concluído.


---
### Relatório

In [13]:
for prefix in sample_prefixes:
    eokp.processar_amostra(prefix, dfs_quant_processed, results_by_sample)

⚠️ Ignorando amostra padrão GT_PadraoHidroc_13-06-2024

🔍 Processando fusão e relatório para GT_OE2-23_13-06-2024...
✅ Fusão completa para GT_OE2-23_13-06-2024
✅ Arquivo salvo: results\GT_OE2-23_13-06-2024_Identificacao_Combinada.xlsx
✅ Relatório Word salvo: results\GT_OE2-23_13-06-2024_relatorio.docx

🔍 Processando fusão e relatório para GT_OE3-23_13-06-2024...
✅ Fusão completa para GT_OE3-23_13-06-2024
✅ Arquivo salvo: results\GT_OE3-23_13-06-2024_Identificacao_Combinada.xlsx
✅ Relatório Word salvo: results\GT_OE3-23_13-06-2024_relatorio.docx

🔍 Processando fusão e relatório para GT_OE1-23_13-06-2024...
✅ Fusão completa para GT_OE1-23_13-06-2024
✅ Arquivo salvo: results\GT_OE1-23_13-06-2024_Identificacao_Combinada.xlsx
✅ Relatório Word salvo: results\GT_OE1-23_13-06-2024_relatorio.docx
